In [84]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

from code_.process_conll import process_file, extract_features

from code_.bert import Tokenizer, convert_to_dataset, compute_metrics, get_labels_list_from_dataset, task, batch_size, model_checkpoint
# from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
# from transformers import DataCollatorForTokenClassification
# from datasets import load_metric

In [85]:
df_val = process_file('data/raw/en_ewt-up-dev.conllu')
df_train = process_file('data/raw/en_ewt-up-train.conllu')
df_test = process_file('data/raw/en_ewt-up-test.conllu')

process_file(): dataframe len: 4979
process_file(): dataframe len: 40498
process_file(): dataframe len: 4802


In [86]:
df_val = extract_features(df_val)
df_train = extract_features(df_train)
df_test = extract_features(df_test)

In [87]:
dataset = convert_to_dataset(df_train, df_val, df_test)

In [88]:
labels_list = get_labels_list_from_dataset(dataset)
print(labels_list)

['R-ARG1', 'ARGM-ADJ', 'C-ARG2', 'ARG1-DSP', 'R-ARG3', 'C-ARGM-PRR', 'R-ARGM-DIR', 'R-ARGM-MNR', 'C-ARG1', 'ARG1', 'ARG4', 'C-V', 'R-ARG2', 'C-ARGM-LOC', 'ARGM-PRR', 'C-ARGM-PRP', 'C-ARG3', 'ARGM-MNR', 'R-ARGM-GOL', 'R-ARGM-COM', 'R-ARGM-CAU', 'ARGM-ADV', 'C-ARGM-ADV', 'V', '_', 'ARG3', 'ARGM-LOC', 'C-ARGM-COM', 'ARGM-CXN', 'ARGA', 'ARGM-LVB', 'ARGM-DIS', 'ARGM-PRD', 'ARGM-DIR', 'C-ARGM-GOL', 'ARGM-TMP', 'ARGM-EXT', 'ARGM-COM', 'C-ARG0', 'R-ARG4', 'ARGM-REC', 'ARGM-NEG', 'ARGM-PRP', 'ARGM-MOD', 'ARG5', 'R-ARGM-ADV', 'C-ARG1-DSP', 'R-ARG0', 'C-ARGM-EXT', 'ARG0', 'R-ARGM-TMP', 'C-ARGM-TMP', 'R-ARGM-LOC', 'C-ARGM-MNR', 'ARGM-GOL', 'ARGM-CAU', 'C-ARG4', 'C-ARGM-CXN', 'C-ARGM-DIR', 'ARG2']


In [89]:
tok = Tokenizer(model_checkpoint, labels_list)

In [90]:
dataset['train'].to_pandas()

,sentences,labels,labels_list
0,"[Al, -, Zaman, :, American, forces, killed, Sh...","_, _, _, _, _, ARG0, V, ARG1, _, _, _, _, _, _...","[_, _, _, _, _, ARG0, V, ARG1, _, _, _, _, _, ..."
1,"[[, This, killing, of, a, respected, cleric, w...","_, _, V, _, _, _, ARG1, _, _, _, _, _, _, _, _...","[_, _, V, _, _, _, ARG1, _, _, _, _, _, _, _, ..."
2,"[[, This, killing, of, a, respected, cleric, w...","_, _, _, _, _, _, _, _, V, _, _, _, _, _, _, _...","[_, _, _, _, _, _, _, _, V, _, _, _, _, _, _, ..."
3,"[[, This, killing, of, a, respected, cleric, w...","_, _, ARG0, _, _, _, _, ARGM-MOD, _, V, ARGM-G...","[_, _, ARG0, _, _, _, _, ARGM-MOD, _, V, ARGM-..."
4,"[[, This, killing, of, a, respected, cleric, w...","_, _, _, _, _, _, _, _, _, _, _, _, _, ARG1, _...","[_, _, _, _, _, _, _, _, _, _, _, _, _, ARG1, ..."
...,...,...,...
40493,"[I, will, never, return, there, again, (, and,...","ARG1, ARGM-MOD, ARGM-NEG, V, ARG4, ARGM-TMP, _...","[ARG1, ARGM-MOD, ARGM-NEG, V, ARG4, ARGM-TMP, ..."
40494,"[I, will, never, return, there, again, (, and,...","_, _, _, _, _, _, _, _, _, V, _, _, ARGM-PRR, ...","[_, _, _, _, _, _, _, _, _, V, _, _, ARGM-PRR,..."
40495,"[I, will, never, return, there, again, (, and,...","ARG0, _, _, _, _, _, _, _, ARGM-TMP, ARGM-LVB,...","[ARG0, _, _, _, _, _, _, _, ARGM-TMP, ARGM-LVB..."
40496,"[I, will, never, return, there, again, (, and,...","_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _...","[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ..."


In [91]:
tok.tokenizer(dataset['validation'][0]['sentences'])

{'input_ids': [[101, 2013, 102], [101, 1996, 102], [101, 9706, 102], [101, 3310, 102], [101, 2023, 102], [101, 2466, 102], [101, 1024, 102], [101, 102, 102], [101, 3310, 102]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [92]:
examples = dataset['validation'][:2]

In [93]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tok.tokenizer(examples["sentences"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["labels_list"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                print(label)
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["enc_labels"] = labels
    return tokenized_inputs

In [94]:
tokenize_and_align_labels(dataset['train'][0])

IndexError: string index out of range

In [50]:
# tokenized_datasets = dataset.map(tok.tokenize_and_align_labels)

Map:   0%|          | 0/40498 [00:00<?, ? examples/s]

ArrowInvalid: Could not convert '_' with type str: tried to convert to int64

In [ ]:
# initialise model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels_list))

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tok.tokenizer)
metric = load_metric("seqeval")

In [ ]:

# labels = [labels_list[i] for i in example[f"{task}_tags"]]
# metric.compute(predictions=[labels], references=[labels])

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tok.tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# train

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('model_checkpoints/some_model_name.pth')

In [ ]:
# results, plots, reports etc.

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results